In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import progressbar
from random import choices

In [100]:
states={}
rx=-5
ro=5
empty=0
startstate="#########"
player=1
df=0.93
alpha=0.05
l=[]
k=0.03
c=0

In [132]:
def addkey(key,player):
    key=key+player
    states[key]=np.random.uniform(-0.01,0.01,9)
    
def checkvalidaction(key,act):
    if key[act]=='#':
        return 1
    else:
        return 0
    
def getnext(state,action,player):
    #print(state," ",action," ",player,"\n")
    l=list(state)
    l[int(action)]=player
    state=''.join(l)
    return state

def getwinner(state):
    l=list(state)
    temp=[['#' for i in range(3)] for j in range(3)]
    for i in range(3):
        for j in range(3):
            temp[i][j]=l[i*3+j]
            #print(temp)
    for i in range(3):
        if temp[i][0]==temp[i][1] and temp[i][1]==temp[i][2] and temp[i][2]!='#':
            return temp[i][0]
        if temp[0][i]==temp[1][i] and temp[1][i]==temp[2][i] and temp[0][i]!='#':
            return temp[0][i]
        
    if temp[0][0]==temp[1][1] and temp[1][1]==temp[2][2] and temp[1][1]!='#':
        return temp[0][0];
    else: 
        if temp[0][2]==temp[1][1] and temp[2][0]==temp[1][1] and temp[2][0]!='#':
            return temp[2][0]
    return 0

def getreward(state):
    winner=getwinner(state)
    if winner=='X':
        return 1
    if winner=='O':
        return -1
    if '#' not in state:
        return 0.5
    return 0

def gameover(state):
    if '#' not in state:
        return 1
    if getwinner(state)!=0:
        return 1
    return 0

def gameoverlast(state):
    if '#' not in state:
        return 1
    return 0
 
def switchplayers(pl):
    if pl=='X':
        return 'O'
    else :
        return 'X'
    
def updateqvalues(state,act,ns,rw,pl):
    ex=0
    if gameoverlast(ns):
        ex=rw
    else :
        if pl=='X':
            ex=rw+(df*np.amin(states[ns+switchplayers(pl)]))
            #print(ex)
        else :
            ex=rw+(df*np.amax(states[ns+switchplayers(pl)]))
    state=state+pl
    #if alpha*(ex-states[state][act])>0.1 and alpha*(ex-states[state][act])<1:
        #print(alpha*(ex-states[state][act]))
    states[state][act]=states[state][act]+alpha*(ex-states[state][act])
    


In [133]:
def chooseaction(key,player):
    if key==startstate:
        return int(random.random()*9)
    ch=int(random.random()*2)
    if ch==1:
        while(1):
            act=int(random.random()*9)
            if checkvalidaction(key,act):
                return act
    else:
        key_t=key+player
        x=np.argsort(states[key_t])
        x=np.flip(x)
        for y in x:
            if checkvalidaction(key,y):
                return y  

In [134]:
def getplacesfilled(state):
    c=0
    for x in state:
        if x=="#":
            c+=1
    return 9-c

def onlypos(state):
    l=list(state)
    return l.index('#')

def getcdf(temp):
    t=[]
    for i in range(len(temp)):
        su=0.0
        for j in range(i):
            su=su+temp[j]
        t.append(su)
    return t

def decodecdf(act,temp):
    for i in range(8,-1,-1):
        if act>temp[i]:
            return i
    return 0

def cact(key,player,k):
    if getplacesfilled(key)==8:
        return onlypos(key)
    val=states[key+player]
    lowest=0
    if player=='X':
        lowest=np.amin(val)
    else :
        val=val*-1
        lowest=np.amin(val)
    if lowest<0:
        lowest=lowest*-1+0.1
        val=val+lowest
    val=np.array([k**x for x in val])
    val=val/(np.sum(val))
    temp=[int(1000*val[i]) for i in range(9)]
    temp=getcdf(temp)
    while(1):
        act=int(random.random()*1000)
        act=decodecdf(act,temp)
        if checkvalidaction(key,int(act)):
            return act
    

In [135]:
def qlearn(k,maxgames):
    games=0
    player='X'
    playerop='O'
    player=int(random.random()*2)
    if player==1:
        player='X'
    else :
        player='O'
    for i in progressbar.progressbar(range(maxgames)):
        key=startstate
        cplayer=player
        while(23):            
            if key+player not in states:
                addkey(key,player)
            action=cact(key,player,k)
            nextstate=getnext(key,action,player)
            reward=getreward(nextstate)
            if nextstate+switchplayers(player) not in states:
                addkey(nextstate,switchplayers(player))
            updateqvalues(key,action,nextstate,reward,player)
            if gameover(nextstate)==1:
                player=switchplayers(cplayer)
                break
            else :
                key=nextstate
                player=switchplayers(player)
        if games%10000==0:
            k+=0.1
    #return k
                        

In [136]:
k=0.15
states={}
qlearn(k,200000)

 67% (134721 of 200000) |##########      | Elapsed Time: 0:01:02 ETA:   0:00:36

KeyboardInterrupt: 

In [137]:
print(len(states))

10956


In [141]:
import pickle
def getdata():
    with open("dataqes.pkl","rb")as f:
        return pickle.load(f)

In [142]:
def getactfromuser(state):
    while(1):
        y=int(input("ROW"))
        x=int(input("COLUMN"))
        if checkvalidaction(state,(y-1)*3+(x-1)):
            state=getnext(state,(y-1)*3+(x-1),'O')
            return state

def getactfromcomp(state):
    x=np.argsort(states[state+'X'])
    x=np.flip(x)
    for y in x:
        if checkvalidaction(state,y):
            state=getnext(state,y,'X')
            return state
        
def printboard(state):
    l=list(state)
    for i in range(3):
        for j in range(3):
            print(l[i*3+j]," ",end='')
        print("\n")
        
import os
from IPython.display import clear_output
def gameplay():
    state=startstate
    for i in range(5):
        state=getactfromuser(state)
        clear_output()
        printboard(state)
        print(states[state+'X'])
        if getwinner(state)=='O':
            print("You Win\n")
            return 1
        if gameover(state):
            return 1
        state=getactfromcomp(state)
        #clear_output()
        printboard(state)
        print(states[state+'O'])
        if getwinner(state)=='X':
            print("Computer wins\n")
            return 1
        
states=getdata()   

In [147]:
gameplay()

O  #  X  

O  O  #  

#  #  X  

[ 0.00553494  0.00389969  0.00303497  0.00224693 -0.00810195  0.59833387
 -0.00905678  0.02528714 -0.00612325]
O  #  X  

O  O  X  

#  #  X  

[-0.00426522  0.00671578  0.00055142 -0.00291428  0.0042693  -0.00026722
 -0.00887015  0.00117055  0.00013898]
Computer wins



1

In [ ]:
def randomgamelay(games):
    for i in range